# Outlier Detection Using Unsupervised Machine Learning

## Introduction to Anomaly Detection with PyOD

Welcome to this hands-on lab on **anomaly detection** using the **Python Outlier Detection (PyOD)** library. This notebook will guide you through various machine learning-based approaches to detect anomalies in time series data.

### What is Anomaly Detection?

**Anomaly detection** (also called outlier detection) is the process of identifying data points, events, or observations that deviate significantly from the majority of the data. Unlike statistical methods that rely on simple thresholds and distributions, machine learning approaches can:

- Handle **high-dimensional data** with multiple features
- Detect **complex patterns** that simple statistics might miss
- Learn **contextual relationships** between features
- Adapt to **non-linear patterns** in the data

### Types of Anomalies

1. **Point Anomalies**: Individual data points that are anomalous (e.g., an unusually high taxi ridership on a random Tuesday)
2. **Contextual Anomalies**: Data points that are anomalous in a specific context (e.g., low ridership on a holiday is normal, but low ridership on a regular workday is anomalous)
3. **Collective Anomalies**: A collection of related data points that are anomalous together (e.g., a sequence of unusual events)

### About PyOD

PyOD (Python Outlier Detection) is a comprehensive Python toolkit for detecting outliers in multivariate data. It provides:
- **30+ algorithms** from different families (statistical, proximity-based, clustering, neural networks)
- A **unified API** similar to scikit-learn
- **Scalable** implementations for large datasets
- Tools for **model evaluation and comparison**

In this lab, we'll explore algorithms from different families and learn when to use each approach.

# Technical Requirements

Before we begin, let's verify that we have all the necessary libraries installed. These libraries form the foundation of our anomaly detection toolkit.

In [3]:
!uv pip install pyod

Using Python 3.12.11 environment at: /Users/tarekatwan/Repos/MyWork/Teach/repos/adv_ml_ds/dev1
Resolved 18 packages in 318ms                                        
Installed 3 packages in 33ms                                
 + llvmlite==0.45.1
 + numba==0.62.1
 + pyod==2.0.5


In [4]:
import matplotlib 
import pandas as pd
import pyod 
import statsmodels

print(f'''
Matplotlib -> {matplotlib.__version__}
pandas -> {pd.__version__}   
''')


Matplotlib -> 3.10.7
pandas -> 2.3.3   



In [5]:
from pyod import version
version.__version__

'2.0.5'

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


plt.rcParams["figure.figsize"] = [12, 5]

## Dataset: NYC Taxi Ridership

We'll use NYC taxi ridership data, which contains the number of passengers over time. This is an excellent dataset for learning anomaly detection because:

- It has **clear seasonal patterns** (daily, weekly, yearly)
- It contains **known anomalies** (holidays, special events)
- It's a **real-world use case** (anomaly detection in transportation systems)

The dataset has observations every 30 minutes, showing how many taxi passengers were recorded during each time window.

In [9]:
file = Path("data/weight-height.csv")
wh = pd.read_csv(file)
wh

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801
...,...,...,...
9995,Female,66.172652,136.777454
9996,Female,67.067155,170.867906
9997,Female,63.867992,128.475319
9998,Female,69.034243,163.852461


# Detecting Outliers Using Distance-Based Algorithms (PyOD)

## What are Distance-Based Algorithms?

Distance-based algorithms identify anomalies by measuring how **far** data points are from their neighbors. The intuition is simple:

- **Normal points** are surrounded by many neighbors (they're in dense regions)
- **Anomalous points** are isolated and far from other points (they're in sparse regions)

These algorithms work well when anomalies are in low-density regions of the feature space.

---

## Algorithm 1: K-Nearest Neighbors (KNN)

### How KNN Works for Anomaly Detection

The **KNN algorithm** identifies outliers based on the distance to their k-nearest neighbors:

1. **For each data point**, find its k closest neighbors
2. **Calculate a distance metric** (e.g., mean, median, or largest distance to the k neighbors)
3. **Assign an anomaly score** based on this distance
4. **Points with large distances** to their neighbors are considered outliers

**Key Hyperparameters:**
- `n_neighbors` (k): How many neighbors to consider (default: 5)
- `method`: How to aggregate distances ('mean', 'median', 'largest')
- `contamination`: Expected proportion of outliers in the data (e.g., 0.03 = 3%)

**When to use KNN:**
- ✅ Simple to understand and interpret
- ✅ Works well when outliers are far from normal points
- ❌ Computationally expensive for large datasets
- ❌ Sensitive to the choice of k

---

## Algorithm 2: Local Outlier Factor (LOF)

### How LOF Works for Anomaly Detection

**LOF** is an improvement over KNN that considers **local density** rather than just distance:

1. **Calculate local density** for each point based on distances to its k neighbors
2. **Compare each point's density** to the density of its neighbors
3. **Compute LOF score**: Ratio of the point's density to its neighbors' average density
4. **Points in low-density regions** relative to their neighbors are outliers

**Key Differences from KNN:**
- LOF considers **relative density** (compares local densities)
- KNN uses **absolute distance** (compares distances directly)
- LOF handles **varying density regions** better than KNN

**Key Hyperparameters:**
- `n_neighbors`: Number of neighbors to use for density estimation (default: 20)
- `contamination`: Expected proportion of outliers

**When to use LOF:**
- ✅ Data has **clusters with different densities**
- ✅ Need to detect **local** anomalies (outliers within a cluster)
- ✅ More robust than KNN for complex data distributions
- ❌ Still computationally expensive for very large datasets

---

Let's apply both algorithms to detect point outliers in our taxi data!

In [10]:
from pyod.models.knn import KNN
from pyod.models.lof import LOF

## Detecting Point Outliers

In this section, we apply KNN and LOF directly to the raw time series values. This approach detects **point anomalies** - individual observations that have unusual values compared to other observations, without considering temporal context or trends.

In [41]:
knn = KNN(contamination=0.005,
          method='mean',
          n_neighbors=5)
print(knn)

KNN(algorithm='auto', contamination=0.005, leaf_size=30, method='mean',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)


In [42]:
lof = LOF(contamination=0.005, 
          n_neighbors=20) 
print(lof)

LOF(algorithm='auto', contamination=0.005, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2)


In [43]:
knn.fit(wh[['Height']])
lof.fit(wh[['Height']])

LOF(algorithm='auto', contamination=0.005, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2)

In [44]:
knn_pred = pd.Series(knn.predict(wh[['Height']]))

lof_pred = pd.Series(lof.predict(wh[['Height']]))

In [45]:
print('Number of KNN outliers = ', knn_pred.sum())
print('Number of LOF outliers = ', lof_pred.sum())

Number of KNN outliers =  28
Number of LOF outliers =  42


# Detecting Outliers Using Clustering-Based Algorithms (PyOD)

## What are Clustering-Based Algorithms?

Clustering-based algorithms use a different intuition than distance-based methods:

**Core Idea**: 
- First, **group similar data points** into clusters
- Then, identify points that **don't fit well** into any cluster
- Points far from cluster centers or in small/sparse clusters are anomalies

**Advantage over distance-based methods**:
- Can handle **complex data distributions** with multiple modes
- More efficient for **large datasets** (cluster first, then evaluate)
- Can detect anomalies as **points between clusters**

---

## Algorithm: Cluster-Based Local Outlier Factor (CBLOF)

### How CBLOF Works

CBLOF combines clustering with outlier scoring:

1. **Clustering Phase**: 
   - Apply clustering (e.g., K-means) to partition the data into `n_clusters` groups
   - Classify clusters as **large** (many points) or **small** (few points)

2. **Outlier Scoring**:
   - For points in **large clusters**: Score based on distance to cluster center
   - For points in **small clusters**: Score based on distance to nearest large cluster
   - Intuition: Small clusters are likely to contain outliers

3. **Final Score Calculation**:
   - Uses `alpha` and `beta` parameters to weight the contribution
   - `alpha`: Threshold to classify clusters as large vs. small (default: 0.9)
   - `beta`: Weight factor for small cluster penalty (default: 5)

### Key Hyperparameters

- **`n_clusters`**: Number of clusters to create
  - Too few: Might group anomalies with normal points
  - Too many: Normal points might form small clusters
  
- **`contamination`**: Expected proportion of outliers (e.g., 0.03 = 3%)

- **`alpha`**: Ratio to determine large vs. small clusters
  - Default 0.9 means: clusters with > 90% of average cluster size are "large"
  
- **`beta`**: Penalty multiplier for small clusters
  - Higher values = stronger penalty for points in small clusters

### When to Use CBLOF

- ✅ Data has **natural groupings** or clusters
- ✅ Need better **scalability** than pure distance-based methods
- ✅ Outliers are likely to be **isolated** or in sparse regions
- ❌ Need to choose the number of clusters (requires domain knowledge)
- ❌ Less effective if data doesn't have clear cluster structure

Let's see how CBLOF performs on our taxi data!

In [33]:
from pyod.models.cblof import CBLOF

In [38]:
cblof = CBLOF(n_clusters=8, 
              contamination=0.001,
              alpha=0.9,
              beta=5)
cblof

CBLOF(alpha=0.9, beta=5, check_estimator=False, clustering_estimator=None,
   contamination=0.001, n_clusters=8, n_jobs=None, random_state=None,
   use_weights=False)

In [39]:
cblof.fit(wh[['Height']])

cblof.predict(wh[['Height']]).sum()

np.int64(10)

# Detecting Outliers Using Probabilistic and Statistical Algorithms

## What are Probabilistic Algorithms?

Unlike distance or clustering methods, probabilistic algorithms model the **probability distribution** of the data:

**Core Idea**:
- Learn the **joint probability distribution** of all features
- Calculate how **probable** each data point is under this distribution
- Points with **low probability** are considered anomalies

**Advantages**:
- ✅ Theoretically grounded in **statistics and probability theory**
- ✅ Can handle **dependencies between features** 
- ✅ Often **parameter-free** or require minimal tuning
- ✅ Fast prediction after training

---

## Algorithm 1: COPOD (Copula-Based Outlier Detection)

### How COPOD Works

COPOD uses **copula theory** from statistics to model dependencies:

1. **Marginal Distributions**: 
   - Model each feature's distribution independently
   - Estimate the **empirical cumulative distribution function (ECDF)** for each feature
   
2. **Tail Probabilities**:
   - For each feature, compute left-tail and right-tail probabilities
   - Left-tail: P(X ≤ x) - how unusual is this low value?
   - Right-tail: P(X ≥ x) - how unusual is this high value?
   
3. **Copula-Based Combination**:
   - Use **copula functions** to combine probabilities across features
   - Captures dependencies between features without assuming specific distributions
   
4. **Outlier Score**:
   - Compute final score based on combined tail probabilities
   - Lower probability = higher anomaly score

### Key Advantages of COPOD

- **Parameter-free**: No hyperparameters to tune (except contamination)
- **Fast**: Linear time complexity O(n)
- **Interpretable**: Based on probability theory
- **Distribution-free**: Doesn't assume Gaussian or any specific distribution

### When to Use COPOD

- ✅ Need a **simple, fast** algorithm with **no tuning**
- ✅ Data has **multiple features** with potential dependencies
- ✅ Want **probabilistic interpretation** of anomaly scores
- ✅ Working with **large datasets** (very efficient)
- ❌ Data is highly complex and non-linear (consider deep learning methods)

---

## Algorithm 2: ECOD (Empirical Cumulative Distribution Outlier Detection)

ECOD is a simpler, even faster variant:

### How ECOD Works

1. **Empirical CDFs**: Compute ECDF for each feature independently
2. **Tail Probabilities**: Calculate left and right tail probabilities
3. **Independence Assumption**: Combines probabilities assuming feature independence
4. **Outlier Score**: Points in the tails of multiple features score highest

### COPOD vs. ECOD

| Feature | COPOD | ECOD |
|---------|-------|------|
| **Speed** | Fast | Even faster |
| **Dependencies** | Models feature dependencies | Assumes independence |
| **Accuracy** | Higher for correlated features | Good for independent features |
| **Complexity** | Uses copula theory | Simple tail probability |

Let's apply both algorithms to see how they perform!

In [ ]:
from pyod.models.copod import COPOD

In [ ]:
copod = COPOD(contamination=0.03)
copod.fit(tx)

In [ ]:
copod_pred = pd.Series(copod.predict(tx), 
                      index=tx.index)

In [ ]:
print('Number of COPOD outliers = ', copod_pred.sum())

In [ ]:
# extract the outlier values and dates from copod_pred
copod_outliers = copod_pred[copod_pred == 1]
copod_outliers = tx.loc[copod_outliers.index]
print(copod_outliers)

In [ ]:
known_outliers

In [ ]:
plot_outliers(copod_outliers, tx, 'COPOD')

In [ ]:
plot_outliers(known_outliers, tx, 'Known')

## Additional Probabilistic Methods: ECOD

In this section, we explore **ECOD** as another probabilistic approach. Compare its results with COPOD to see how the independence assumption affects anomaly detection!

In [ ]:
from pyod.models.ecod import ECOD

# Initialize and fit the ECOD model
ecod = ECOD(contamination=0.03)
ecod.fit(tx)

# Predict outliers
ecod_pred = pd.Series(ecod.predict(tx), 
                     index=tx.index)
print('Number of ECOD outliers = ', ecod_pred.sum())

# extract the outlier values and dates from ecod_pred
ecod_outliers = ecod_pred[ecod_pred == 1]
ecod_outliers = tx.loc[ecod_outliers.index]
print(ecod_outliers)

In [ ]:
plot_outliers(ecod_outliers, tx, 'ECOD')

# Detecting Outliers Using Kernel-Based Algorithms (PyOD)

## What are Kernel-Based Methods?

Kernel-based algorithms use **kernel functions** to transform data into higher-dimensional spaces where separation between normal and anomalous points becomes easier.

**Core Concept**:
- Data that's **not linearly separable** in the original space...
- ...might be **separable in a transformed (kernel) space**
- Use the **"kernel trick"** to work in high dimensions without explicitly computing the transformation

---

## Algorithm: One-Class SVM (OCSVM)

### How One-Class SVM Works

Unlike traditional SVM (which learns a boundary between two classes), **One-Class SVM** learns a boundary around **one class** (normal data):

1. **Map to Feature Space**:
   - Use a kernel function (RBF, linear, polynomial, sigmoid) to implicitly transform data
   - In this new space, find a **hyperplane** that separates normal data from the origin

2. **Learn the Decision Boundary**:
   - Fit a hyperplane that **encloses most normal points**
   - Points are allowed to fall outside based on the `nu` parameter
   - The hyperplane maximizes margin while allowing some flexibility

3. **Classify New Points**:
   - Points **inside the boundary** → normal
   - Points **outside the boundary** → anomalies

### Key Hyperparameters

1. **`kernel`**: The kernel function to use
   - **'rbf' (Radial Basis Function)**: Most popular, handles non-linear patterns
     - Creates smooth, circular decision boundaries
   - **'linear'**: Fastest, for linearly separable data
   - **'poly'**: Polynomial boundaries, can overfit
   - **'sigmoid'**: Neural network-like transformation

2. **`gamma`**: Kernel coefficient (for 'rbf', 'poly', 'sigmoid')
   - **'auto'**: Uses 1 / n_features
   - **High gamma**: Model focuses on nearby points (complex, can overfit)
   - **Low gamma**: Model considers distant points (simpler, smoother boundary)

3. **`nu`**: Upper bound on fraction of outliers and lower bound on fraction of support vectors
   - Similar to contamination, but interpreted differently
   - Range: (0, 1), typically 0.5
   - Higher nu → more flexible boundary, more points classified as outliers

### The Importance of Scaling

⚠️ **Critical**: OCSVM is **sensitive to feature scales**!

- Features with large ranges dominate the distance calculations
- **Always standardize/normalize** features before applying OCSVM
- Use `StandardScaler` or PyOD's `standardizer` utility

### When to Use OCSVM

- ✅ Data has **complex, non-linear patterns**
- ✅ Need a **robust** decision boundary
- ✅ Have **only normal data** for training (one-class problem)
- ✅ Can afford to **scale your features** properly
- ❌ Have very large datasets (can be slow)
- ❌ Need easy interpretability (kernel methods are "black box")

### Kernel Selection Guide

- **Start with 'rbf'**: Works well in most cases
- **Try 'linear'**: If data seems linearly separable or for baseline
- **Use 'poly'**: If you suspect polynomial relationships
- **Avoid 'sigmoid'**: Unless you have specific reasons

Let's see how OCSVM performs with different kernels!

In [ ]:
from pyod.models.ocsvm import OCSVM

In [ ]:
ocsvm = OCSVM(contamination=0.03, 
              kernel='rbf',
              gamma='auto',
              nu=0.5)
ocsvm.fit(tx)

In [ ]:
ocsvm_pred = pd.Series(ocsvm.predict(tx), 
                      index=tx.index)

In [ ]:
print('Number of OCSVM outliers = ', ocsvm_pred.sum())

In [ ]:
# extract the outlier values and dates from ocsvm_pred
ocsvm_outliers = ocsvm_pred[ocsvm_pred == 1]
ocsvm_outliers = tx.loc[ocsvm_outliers.index] 
print(ocsvm_outliers)

In [ ]:
print(known_outliers)

In [ ]:
plot_outliers(known_outliers, tx, 'Known')

In [ ]:
plot_outliers(ocsvm_outliers, tx, 'OCSVM')

In [ ]:
from pyod.utils.utility import standardizer

ocsvm = OCSVM(contamination=0.03, 
              kernel='rbf',
              gamma='auto',
              nu=0.5)

scaled = standardizer(tx)
ocsvm.fit(scaled)
ocsvm_pred_sc = pd.Series(ocsvm.predict(scaled), 
                      index=tx.index)

# extract the outlier values and dates from ocsvm_pred_sc
ocsvm_outliers_sc = ocsvm_pred_sc[ocsvm_pred_sc == 1]
ocsvm_outliers_sc = tx.loc[ocsvm_outliers_sc.index] 
print(ocsvm_outliers_sc)

In [ ]:
plot_outliers(ocsvm_outliers_sc, tx, 'OCSVM after scaling')

## Exploring Different Kernels

Different kernels can capture different types of patterns. Let's compare how 'linear', 'poly', 'rbf', and 'sigmoid' kernels detect anomalies in our taxi data.

**Watch for**:
- Which kernel finds the most known anomalies?
- Which kernel produces the most stable results?
- How do the decision boundaries differ?

In [ ]:
for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
    ocsvm = OCSVM(contamination=0.03, 
                  kernel=kernel)
    ocsvm.fit(scaled)
    ocsvm_pred_sc = pd.Series(ocsvm.predict(scaled), 
                              index=tx.index, 
                              name=kernel)
    ocsvm_outliers_sc = ocsvm_pred_sc[ocsvm_pred_sc == 1]
    ocsvm_outliers_sc = tx.loc[ocsvm_outliers_sc.index]
    print(f"Outliers using {kernel} kerenl: \n{ocsvm_outliers_sc}")
    #plot_outliers(ocsvm_outliers_sc, tx, f"OCSVM using {kernel} kernel")

# Detecting Outliers Using Ensemble Methods (PyOD)

## What are Ensemble Methods?

**Ensemble methods** combine multiple models or decision strategies to improve overall performance:

**Core Principle**: "Wisdom of the crowd"
- Individual models might make mistakes
- But **combining multiple models** often produces better results
- Different models capture different aspects of anomalies

**Types of Ensembles**:
1. **Bagging**: Train multiple models on random subsets of data
2. **Boosting**: Train models sequentially, focusing on hard cases
3. **Feature bagging**: Train models on random subsets of features
4. **Model combination**: Combine different algorithm types

---

## Algorithm 1: Isolation Forest (IForest)

### How Isolation Forest Works

IForest uses a unique approach based on **isolation** rather than distance or density:

**Key Insight**: 
- **Anomalies are rare and different** → easier to isolate
- **Normal points are common and similar** → harder to isolate

### The Algorithm

1. **Build Isolation Trees** (`n_estimators` trees):
   - Randomly select a feature
   - Randomly select a split value between min and max
   - Split the data recursively
   - Stop when each point is isolated or tree reaches max depth

2. **Measure Path Length**:
   - For each point, count how many splits needed to isolate it
   - **Anomalies** → isolated quickly → **short paths**
   - **Normal points** → many splits needed → **long paths**

3. **Aggregate Across Trees**:
   - Average path length across all trees
   - Normalize to get anomaly score
   - Shorter average path = higher anomaly score

### Visual Intuition

Imagine randomly drawing lines to separate points:
- An outlier sitting far from others gets separated in 1-2 cuts ✂️
- A normal point in a dense cluster needs many cuts ✂️✂️✂️✂️✂️

### Key Hyperparameters

1. **`n_estimators`**: Number of isolation trees (default: 100)
   - More trees → more stable, but slower
   - Typical range: 50-200

2. **`contamination`**: Expected proportion of outliers

3. **`bootstrap`**: Whether to use sampling with replacement
   - `True`: Each tree trained on a bootstrapped sample (introduces more diversity)
   - `False`: Each tree sees all data (faster, but less diverse)

4. **`max_samples`**: Number of samples to draw for each tree
   - Default: 256 or size of dataset if smaller
   - Larger → more accurate but slower
   - Smaller → faster but less stable

### Why Isolation Forest is Popular

- ✅ **Fast**: Linear time complexity O(n)
- ✅ **Scalable**: Works well with large datasets
- ✅ **Few parameters**: Easy to use
- ✅ **No need for distance/density computation**: Different paradigm
- ✅ **Handles high-dimensional data** well
- ✅ **Interpretable**: Path length has intuitive meaning

### When to Use IForest

- ✅ **Large datasets** (very efficient)
- ✅ **High-dimensional data** (doesn't suffer from curse of dimensionality)
- ✅ Need **fast training and prediction**
- ✅ Want a **parameter-free** method (works well with defaults)
- ✅ Outliers are **globally anomalous** (not local anomalies)

---

## Algorithm 2: Deep Isolation Forest (DIF)

### What is DIF?

**DIF** is an advanced variant that combines:
- Isolation Forest's isolation principle
- Deep learning's representation learning

### How DIF Improves on IForest

1. **Feature Representation**:
   - Uses neural networks to learn better feature representations
   - Can capture **non-linear relationships** automatically

2. **Ensemble of Representations**:
   - Each tree uses a different learned representation
   - More diverse ensemble → better performance

### When to Use DIF

- ✅ Data has **complex, non-linear patterns**
- ✅ Have **sufficient training data** for deep learning
- ✅ Standard IForest doesn't perform well
- ❌ Need fast training (DIF is slower than IForest)
- ❌ Need interpretability (neural networks are less interpretable)

Let's apply both IForest and DIF to our taxi data!

In [ ]:
from pyod.models.iforest import IForest

In [ ]:
iforest = IForest(contamination=0.03,
                 n_estimators=100,
                 bootstrap=False,
                 random_state=45)
iforest.fit(tx)

In [ ]:
iforest_pred = pd.Series(iforest.predict(tx), 
                      index=tx.index)

In [ ]:
print('Number of IForest outliers = ', iforest_pred.sum())

In [ ]:
# extract the outlier values and dates from iforest_pred
iforest_outliers = iforest_pred[iforest_pred == 1]
iforest_outliers = tx.loc[iforest_outliers.index] 
print(iforest_outliers)

In [ ]:
plot_outliers(iforest_outliers, tx, 'IForest')

## Advanced Ensemble: Deep Isolation Forest (DIF)

Now let's explore DIF, which combines isolation with deep learning for potentially better anomaly detection on complex patterns.

In [ ]:
import pandas as pd
from pyod.models.dif import DIF
import warnings
warnings.filterwarnings('ignore')

# Initialize and fit the DIF model 
dif = DIF(contamination=0.03, n_estimators=100)
dif.fit(tx)

# Get outliers
dif_scores = dif.decision_scores_
dif_threshold = np.quantile(dif_scores, 0.97)  # Adjust percentile as needed
dif_outliers = tx[dif_scores > dif_threshold]
print(dif_outliers)

In [ ]:
plot_outliers(dif_outliers, tx, 'DIF')

# Detecting Outliers Using Deep Learning (PyOD)

## What are Deep Learning-Based Methods?

Deep learning brings **neural networks** to anomaly detection, offering powerful capabilities:

**Core Idea**:
- Use neural networks to **learn compressed representations** of normal data
- Points that **reconstruct poorly** from these representations are anomalies
- The network learns what "normal" looks like automatically

**Advantages**:
- ✅ **Automatic feature learning**: No manual feature engineering needed
- ✅ **Handle complex patterns**: Can model highly non-linear relationships
- ✅ **Scalable**: Works with high-dimensional data
- ✅ **Flexible**: Can incorporate various architectures

**Challenges**:
- ❌ **Computationally expensive**: Requires more time and resources
- ❌ **Hyperparameter tuning**: Learning rate, epochs, batch size, architecture
- ❌ **Less interpretable**: "Black box" compared to simpler methods
- ❌ **May overfit**: Especially with small datasets

---

## Algorithm 1: AutoEncoder

### What is an AutoEncoder?

An **AutoEncoder** is a neural network trained to **reconstruct its input**:

```
Input → Encoder → Compressed Representation (Bottleneck) → Decoder → Reconstructed Output
```

### How AutoEncoders Detect Anomalies

1. **Training Phase** (on normal data):
   - **Encoder**: Compresses input to a low-dimensional representation (bottleneck)
   - **Decoder**: Reconstructs input from the compressed representation
   - **Objective**: Minimize reconstruction error for normal points
   - The network learns to **encode patterns of normal data**

2. **Detection Phase**:
   - Pass new data through the trained AutoEncoder
   - Calculate **reconstruction error** = |input - reconstructed output|
   - **Normal points**: Small reconstruction error (network learned their patterns)
   - **Anomalous points**: Large reconstruction error (network never learned these patterns)

### Architecture Components

```
Input Layer (n features)
    ↓
Hidden Layer 1 (decreasing size)
    ↓
Hidden Layer 2 (even smaller)
    ↓
Bottleneck (smallest - the compressed representation)
    ↓
Hidden Layer 3 (increasing size)
    ↓
Hidden Layer 4 (same size as input)
    ↓
Output Layer (n features - reconstruction)
```

### Key Hyperparameters

1. **`lr` (learning rate)**: How fast the network learns
   - Too high: Network doesn't converge, unstable learning
   - Too low: Training is very slow
   - Typical range: 0.001 - 0.01

2. **`epoch_num`**: Number of complete passes through the data
   - Too few: Network doesn't learn enough (underfitting)
   - Too many: Network memorizes even anomalies (overfitting)
   - Start with 10-50, increase if needed
   - Use validation to find optimal value

3. **`batch_size`**: Number of samples per training update
   - Smaller: More updates, noisier gradient, can escape local minima
   - Larger: Faster training, more stable gradient, more memory
   - Typical values: 16, 32, 64, 128

4. **Architecture** (hidden layers, neurons):
   - Deeper: Can learn more complex patterns
   - Bottleneck size: Should be much smaller than input (compression)

### Training Tips

**Underfitting Signs**:
- High reconstruction error even on normal data
- Solution: Train longer (more epochs), add capacity (more layers/neurons)

**Overfitting Signs**:
- Training error very low, but poor anomaly detection
- Network reconstructs anomalies well (memorized them)
- Solution: Early stopping, regularization, more training data

### When to Use AutoEncoders

- ✅ **High-dimensional data** (e.g., images, text, many features)
- ✅ Data has **complex, non-linear patterns**
- ✅ Have **sufficient normal training data**
- ✅ Can afford **longer training time**
- ❌ Small datasets (simpler methods might work better)
- ❌ Need fast, real-time detection without GPU
- ❌ Need interpretable results

---

## Algorithm 2: Variational AutoEncoder (VAE)

### What is a VAE?

**VAE** is a probabilistic variant of AutoEncoder:

**Key Differences**:
1. **Probabilistic encoding**: Instead of a single compressed representation, VAE learns a **distribution** (mean and variance)
2. **Regularization**: Forces the learned distribution to be close to a standard normal distribution
3. **Generative**: Can generate new samples (not just reconstruct)

### How VAE Detects Anomalies

1. **Encoder**: Maps input to a probability distribution (μ, σ)
2. **Sampling**: Sample from this distribution (with reparameterization trick)
3. **Decoder**: Reconstructs from the sample
4. **Anomaly Score**: Combination of:
   - Reconstruction error (like AutoEncoder)
   - KL divergence (how far the learned distribution is from standard normal)

### VAE vs. AutoEncoder

| Feature | AutoEncoder | VAE |
|---------|-------------|-----|
| **Output** | Deterministic | Probabilistic |
| **Regularization** | Optional | Built-in (KL divergence) |
| **Overfitting** | More prone | More robust |
| **Anomaly Score** | Reconstruction error | Reconstruction + KL |
| **Complexity** | Simpler | More complex |
| **Performance** | Good | Often better, more robust |

### When to Use VAE over AutoEncoder

- ✅ Want more **robust** anomaly detection
- ✅ Need **generative capabilities**
- ✅ Have **diverse** anomaly types
- ✅ Want built-in **regularization**

Let's apply both AutoEncoder and VAE to see how deep learning performs!

In [ ]:
# !pip install torch
# !pip install tqdm

In [ ]:
from pyod.models.auto_encoder import AutoEncoder

In [ ]:
# lr: Learning rate for optimization
# epoch_num: Number of training iterations
# batch_size: Number of samples per training batch
auto_encoder = AutoEncoder(contamination=0.03,
                           lr=0.001,
                           epoch_num=10,
                           batch_size=32)
auto_encoder.fit(tx)

In [ ]:
ae_predicted = pd.Series(auto_encoder.predict(tx), 
                      index=tx.index)

In [ ]:
print('Number of AutoEncoder outliers = ', ae_predicted.sum())

In [ ]:
# extract the outlier values and dates from ae_predicted
ae_outliers = ae_predicted[ae_predicted == 1]
ae_outliers = tx.loc[ae_outliers.index] 
print(ae_outliers)

In [ ]:
ae_scores = pd.Series(auto_encoder.decision_scores_, 
                      index=tx.index)

threshold = auto_encoder.threshold_
# or you can do it using quantile 
threshold = ae_scores.quantile(0.97)

In [ ]:
ae_outliers =tx[ae_scores > ae_scores.quantile(0.97)]
print(ae_outliers)

In [ ]:
plot_outliers(ae_outliers, tx, 'AutoEncoder')

In [ ]:
%%time
auto_encoder = AutoEncoder(contamination=0.03,
                           lr=0.001,
                           epoch_num=1000,
                           batch_size=32)
auto_encoder.fit(tx)
ae_predicted = pd.Series(auto_encoder.predict(tx), 
                      index=tx.index)
# extract the outlier values and dates from ae_predicted
ae_outliers = ae_predicted[ae_predicted == 1]
ae_outliers = tx.loc[ae_outliers.index] 
print(ae_outliers)

In [ ]:
plot_outliers(ae_outliers, tx, 'AutoEncoder')

## Advanced Deep Learning: Variational AutoEncoder (VAE)

Now let's explore **VAE**, a more sophisticated variant that often provides more robust anomaly detection through its probabilistic approach.

**Watch for**:
- Does VAE find different anomalies than the standard AutoEncoder?
- How does the probabilistic approach affect the results?
- Which deep learning method works better for our taxi data?

In [ ]:
from pyod.models.vae import VAE

# lr: Learning rate for optimization
# epoch_num: Number of training iterations
# batch_size: Number of samples per training batch
vae = VAE(contamination=0.03,
                           lr=0.001,
                           epoch_num=1000,
                           batch_size=32)
vae.fit(tx)

vae_predicted = pd.Series(vae.predict(tx), 
                      index=tx.index)
# extract the outlier values and dates from ae_predicted
vae_outliers = vae_predicted[vae_predicted == 1]
vae_outliers = tx.loc[vae_outliers.index] 
print(vae_outliers)

# Summary and Key Takeaways

## Algorithms We Explored

Throughout this lab, we explored **8+ different anomaly detection algorithms** across multiple families:

### 1. **Distance-Based Algorithms**
- **KNN (K-Nearest Neighbors)**: Uses average/median/max distance to k neighbors
  - Simple, interpretable, but computationally expensive
  - Works well when outliers are far from normal points

- **LOF (Local Outlier Factor)**: Uses local density comparison
  - Better than KNN for data with varying density
  - Detects local anomalies within clusters

### 2. **Clustering-Based Algorithms**
- **CBLOF (Cluster-Based Local Outlier Factor)**: Clusters first, then scores
  - Efficient for large datasets
  - Works well when data has natural groupings
  - Points in small clusters or far from clusters are anomalies

### 3. **Probabilistic/Statistical Algorithms**
- **COPOD (Copula-Based Outlier Detection)**: Uses copula theory
  - Parameter-free (except contamination)
  - Fast, handles feature dependencies
  - Based on tail probabilities

- **ECOD (Empirical Cumulative Outlier Detection)**: Simpler than COPOD
  - Even faster, assumes feature independence
  - Good baseline method

### 4. **Kernel-Based Algorithms**
- **OCSVM (One-Class SVM)**: Learns decision boundary using kernels
  - Handles non-linear patterns through kernel trick
  - Requires feature scaling
  - Multiple kernel options (rbf, linear, poly, sigmoid)

### 5. **Ensemble Methods**
- **Isolation Forest**: Uses isolation trees to detect outliers
  - Very fast and scalable
  - Works well with high-dimensional data
  - Based on "ease of isolation" principle

- **DIF (Deep Isolation Forest)**: Combines isolation with deep learning
  - More sophisticated than IForest
  - Can learn better representations

### 6. **Deep Learning Methods**
- **AutoEncoder**: Neural network that learns to reconstruct normal data
  - Detects anomalies through reconstruction error
  - Good for complex, high-dimensional patterns
  - Requires tuning (learning rate, epochs, batch size)

- **VAE (Variational AutoEncoder)**: Probabilistic variant of AutoEncoder
  - More robust than standard AutoEncoder
  - Uses both reconstruction error and KL divergence
  - Better regularization through probabilistic approach

---

## Techniques for Better Anomaly Detection

### 1. **Time Series Decomposition**
- Separate trend, seasonality, and residuals
- Apply anomaly detection to residuals
- Removes seasonal patterns that could hide true anomalies

### 2. **Sliding Windows**
- Create sequences of consecutive observations
- Detect patterns in temporal context
- Good for contextual anomalies

### 3. **Feature Engineering**
- Add temporal features (day of week, month, year)
- Use cyclical encoding for periodic features
- Incorporate domain knowledge (holidays, weekends)
- Combine with sliding windows for rich representations

### 4. **Feature Scaling**
- Essential for distance-based and kernel methods
- Use StandardScaler or PyOD's standardizer
- Not needed for tree-based methods (IForest)

---

## Choosing the Right Algorithm

| Scenario | Recommended Algorithm |
|----------|----------------------|
| **Large dataset, need speed** | Isolation Forest, COPOD, ECOD |
| **Small dataset** | KNN, LOF, OCSVM |
| **High-dimensional data** | Isolation Forest, AutoEncoder, VAE |
| **Complex non-linear patterns** | OCSVM (RBF), AutoEncoder, VAE |
| **Varying density clusters** | LOF, CBLOF |
| **Need interpretability** | KNN, Isolation Forest |
| **Parameter-free method** | COPOD, ECOD |
| **Time series with seasonality** | Use decomposition + any algorithm |
| **Contextual anomalies** | Sliding windows + feature engineering |

---

## PyOD Workflow Best Practices

1. **Initialize** the model with hyperparameters
   ```python
   model = KNN(contamination=0.03, n_neighbors=5)
   ```

2. **Fit** the model on data
   ```python
   model.fit(X)
   ```

3. **Get predictions** (binary labels: 0=normal, 1=anomaly)
   ```python
   predictions = model.predict(X)
   ```

4. **Get decision scores** (continuous scores)
   ```python
   scores = model.decision_scores_
   # or
   scores = model.decision_function(X)
   ```

5. **Get probabilities** (if needed)
   ```python
   proba = model.predict_proba(X, method='linear')
   ```

6. **Save/Load model**
   ```python
   from joblib import dump, load
   dump(model, 'model.joblib')
   model = load('model.joblib')
   ```

---

## Key Parameters

- **`contamination`**: Expected proportion of outliers (e.g., 0.03 = 3%)
  - Sets the threshold for classification
  - Critical parameter for all algorithms
  - Should be based on domain knowledge

- **`n_neighbors` (KNN, LOF)**: Number of neighbors to consider
  - Small k: More sensitive to local variations
  - Large k: Smoother, more global view

- **`n_clusters` (CBLOF)**: Number of clusters
  - Domain-dependent
  - Try different values and evaluate

- **`kernel`, `gamma`, `nu` (OCSVM)**: Control decision boundary
  - Start with 'rbf' kernel and 'auto' gamma
  - Adjust based on results

- **`n_estimators` (IForest)**: Number of trees
  - More trees = more stable, but slower
  - 100 is usually sufficient

- **`lr`, `epoch_num`, `batch_size` (AutoEncoder, VAE)**: Training parameters
  - Require experimentation
  - Use validation to find optimal values

---

## Evaluation Strategies

1. **Use known anomalies** (if available) as ground truth
2. **Visual inspection**: Plot detected anomalies
3. **Multiple algorithms**: Compare results across different methods
4. **Domain expertise**: Verify if detected anomalies make sense
5. **PyOD's evaluation utilities**:
   ```python
   from pyod.utils import evaluate_print
   evaluate_print('ModelName', y_true, decision_scores)
   ```

---

## Final Thoughts

**No single algorithm is best for all cases!**

- Start with **simple, fast methods** (COPOD, IForest)
- Try **multiple algorithms** and compare
- Consider the **specific characteristics** of your data
- Combine **domain knowledge** with algorithmic results
- Use **feature engineering** to improve detection
- **Visualize results** to build intuition

**The best approach is often**:
1. Try several algorithms
2. Ensemble their results (e.g., voting, averaging scores)
3. Incorporate domain knowledge
4. Iterate based on feedback

